# Wisconsin Real Estate Data Analysis
**Objective:** Explore and clean Wisconsin real estate sales data (2022) and prepare it for visualization. This notebook loads data into SQLite, applies cleaning steps, exports cleaned CSVs.

## Notebook overview
This notebook follows these steps:
1. Load raw CSV into pandas
2. Stage into a local SQLite DB for SQL-based cleaning
3. Remove duplicates and handle missing/invalid values
4. Export cleaned CSVs for use in Tableau

### 1) Import libraries
Import required Python packages. Add additional packages to `requirements.txt` if needed.

In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
import sqlite3 as sql

### 2) Load the raw data
Read the source CSV into a pandas DataFrame and show a quick preview. Verify that the file `2022-property-sales-data.csv` exists in the notebook folder.

In [ ]:
df = pd.read_csv("2022-property-sales-data.csv")
print(df.head())
print("Data shape:", df.shape)

   PropertyID       PropType    taxkey                Address CondoProject  \
0       98461  Manufacturing  30131000   9434-9446 N 107TH ST          NaN   
1       98464     Commercial  30152000   9306-9316 N 107TH ST          NaN   
2       98508    Residential  49980110         9327 N SWAN RD          NaN   
3       98519    Residential  49993200  9411 W COUNTY LINE RD          NaN   
4       98561    Residential  50042000        9322 N JOYCE AV          NaN   

   District  nbhd                      Style         Extwall  Stories  \
0         9  6300           Service Building  Concrete Block      1.0   
1         9  6202  Office Building - 1 Story           Brick      1.0   
2         9    40                        NaN             NaN      NaN   
3         9    40                      Ranch  Aluminum/Vinyl      1.0   
4         9    40                      Ranch  Aluminum/Vinyl      1.0   

   Year_Built  Rooms  FinishedSqft  Units  Bdrms  Fbath  Hbath  Lotsize  \
0      1978.0    

### 3) Create a local SQLite engine and stage the data
Createing a SQLite database file (`real_estate_trends.db`) and writing the DataFrame to a `sales_staging` table. Staging allows using SQL for some transformations and explorations.

In [5]:
engine = create_engine('sqlite:///real_estate_trends.db')

Write the DataFrame into the `sales_staging` table. If the table already exists it will be replaced. This gives us a SQL-friendly copy to run transformations with `%%sql` magic.

In [ ]:

df.to_sql('sales_staging', engine, if_exists='replace', index=False)
print("Wrote table 'sales_staging' with shape:", df.shape)

Wrote table 'sales_staging' with shape: (7146, 20)


In [7]:
df.head()

,PropertyID,PropType,taxkey,Address,CondoProject,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
0,98461,Manufacturing,30131000,9434-9446 N 107TH ST,NaN,9,6300,Service Building,Concrete Block,1.0,1978.0,NaN,20600.0,6,NaN,0,0,0,2022-04-01,950000.0
1,98464,Commercial,30152000,9306-9316 N 107TH ST,NaN,9,6202,Office Building - 1 Story,Brick,1.0,1982.0,NaN,9688.0,23,NaN,0,0,35719,2022-10-07,385000.0
2,98508,Residential,49980110,9327 N SWAN RD,NaN,9,40,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,1341648,2022-01-07,800000.0
3,98519,Residential,49993200,9411 W COUNTY LINE RD,NaN,9,40,Ranch,Aluminum/Vinyl,1.0,1959.0,6.0,1334.0,1,3.0,1,1,83200,2022-08-09,280000.0
4,98561,Residential,50042000,9322 N JOYCE AV,NaN,9,40,Ranch,Aluminum/Vinyl,1.0,1980.0,10.0,1006.0,1,6.0,1,0,8303,2022-05-23,233100.0


### 4) SQL magic 
Loading IPython SQL magic to run SQL queries inline. This is convenient for quick explorations and transformations.

In [8]:
%load_ext sql
%sql sqlite:///real_estate_trends.db

In [9]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'


In [10]:
%%sql
SELECT *
FROM sales_staging LIMIT 5;

 * sqlite:///real_estate_trends.db
Done.


PropertyID,PropType,taxkey,Address,CondoProject,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
98461,Manufacturing,30131000,9434-9446 N 107TH ST,None,9,6300,Service Building,Concrete Block,1.0,1978.0,None,20600.0,6,None,0,0,0,2022-04-01,950000.0
98464,Commercial,30152000,9306-9316 N 107TH ST,None,9,6202,Office Building - 1 Story,Brick,1.0,1982.0,None,9688.0,23,None,0,0,35719,2022-10-07,385000.0
98508,Residential,49980110,9327 N SWAN RD,None,9,40,None,None,None,None,None,None,0,None,0,0,1341648,2022-01-07,800000.0
98519,Residential,49993200,9411 W COUNTY LINE RD,None,9,40,Ranch,Aluminum/Vinyl,1.0,1959.0,6.0,1334.0,1,3.0,1,1,83200,2022-08-09,280000.0
98561,Residential,50042000,9322 N JOYCE AV,None,9,40,Ranch,Aluminum/Vinyl,1.0,1980.0,10.0,1006.0,1,6.0,1,0,8303,2022-05-23,233100.0


### 5) Drop unused columns
Dropping columns that are not needed. 

In [ ]:
%%sql
ALTER TABLE sales_staging DROP Column CondoProject;
ALTER TABLE sales_staging DROP Column taxkey;

 * sqlite:///real_estate_trends.db
Done.
Done.


[]

### 6) Find duplicates
Identifying duplicate rows by partitioning on (`PropertyID`, `Sale_date`). Rows with `row_num > 1` are considered duplicates.

In [12]:
%%sql
WITH sales_cte AS (
  SELECT *,
  ROW_NUMBER() OVER(PARTITION BY PropertyID, Sale_date) as row_num 
  FROM sales_staging )
SELECT * 
FROM sales_cte
WHERE row_num >1;

 * sqlite:///real_estate_trends.db
Done.


PropertyID,PropType,Address,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price,row_num
99593,Condominium,"9051 N 95TH ST, Unit f",9,5060,Condo Townhouse,None,2.5,1970.0,5.0,1054.0,1,3.0,1,1,1,2022-09-29,62500.0,2
215441,Residential,814-816 W WINDLAKE AV,12,4100,Triplex,Wood,2.0,1900.0,14.0,2691.0,3,6.0,3,0,2408,2022-01-31,151000.0,2


Deleting duplicate rows found earlier. M

In [13]:
%%sql
DELETE 
FROM sales_staging 
WHERE rowid IN
(SELECT rowid FROM(
  SELECT rowid,
  ROW_NUMBER() OVER(PARTITION BY PropertyID, Sale_date) as row_num 
  FROM sales_staging)
WHERE row_num >1);

 * sqlite:///real_estate_trends.db
2 rows affected.


[]

In [14]:
%%sql
PRAGMA table_info(sales_staging);

 * sqlite:///real_estate_trends.db
Done.


cid,name,type,notnull,dflt_value,pk
0,PropertyID,BIGINT,0,None,0
1,PropType,TEXT,0,None,0
2,Address,TEXT,0,None,0
3,District,BIGINT,0,None,0
4,nbhd,BIGINT,0,None,0
5,Style,TEXT,0,None,0
6,Extwall,TEXT,0,None,0
7,Stories,FLOAT,0,None,0
8,Year_Built,FLOAT,0,None,0
9,Rooms,FLOAT,0,None,0


### 7) Make the table use strict data types

SQLite is flexible with column types, so it might not store data exactly as numbers or text. To fix this, I am creating a new table called new_table with clear column types, and inserting the data from sales_staging after converting each value to the right type. This makes sure numeric columns are actually saved as numbers.

In [15]:
%%sql
CREATE TABLE new_table (
        PropertyID BIGINT,
    PropType TEXT,
    Address TEXT,
    District INTEGER,
    nbhd INTEGER,
    Style TEXT,
    Extwall TEXT,
    Stories INTEGER,
    Year_Built INTEGER,
    Rooms INTEGER,
    FinishedSqft FLOAT,
    Units INTEGER,
    Bdrms INTEGER,
    Fbath INTEGER,
    Hbath INTEGER,
    Lotsize FLOAT,
    Sale_date TEXT,
    Sale_price FLOAT
);

 * sqlite:///real_estate_trends.db
Done.


[]

In [16]:
%%sql
INSERT INTO new_table 
SELECT 
    PropertyID,
    PropType,
    Address,
    District,
    nbhd,
    Style,
    Extwall,
    CAST(Stories AS INTEGER),
    CAST(Year_Built AS INTEGER),
    CAST(Rooms AS INTEGER),
    FinishedSqft,
    Units,
    CAST(Bdrms AS INTEGER),
    Fbath,
    Hbath,
    Lotsize,
    Sale_date,
    Sale_price
FROM sales_staging;

 * sqlite:///real_estate_trends.db
7144 rows affected.


[]

In [17]:
%%sql
DROP TABLE sales_staging;
ALTER TABLE new_table RENAME TO sales_staging;

 * sqlite:///real_estate_trends.db
Done.
Done.


[]

### 8) Handling missing values
Inspecting critical columns for NULL or empty values and remove rows that lack essential information (Year_Built, FinishedSqft, Sale_price, Rooms).

In [18]:
%%sql
SELECT *
FROM sales_staging
WHERE Year_Built IS NULL OR
TRIM(Year_Built) ='';

 * sqlite:///real_estate_trends.db
Done.


PropertyID,PropType,Address,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
98508,Residential,9327 N SWAN RD,9,40,None,None,None,None,None,None,0,None,0,0,1341648.0,2022-01-07,800000.0
102602,Residential,8201 N 111TH ST,9,240,None,None,None,None,None,None,0,None,0,0,15427.0,2022-05-25,57500.0
103040,Manufacturing,8219 W BRADLEY RD,9,6300,Special - to use this code see supv.,None,None,None,None,None,0,None,0,0,0.0,2022-10-06,820000.0
110027,Commercial,8526 W MILL RD,9,6449,"Licensed Res Care Facility, Halfway House, etc.",None,None,None,None,17095.0,41,None,0,0,74618.0,2022-06-10,1156200.0
135165,Manufacturing,3025 W ATKINSON AV,1,6300,Special - to use this code see supv.,None,None,None,None,None,0,None,0,0,0.0,2022-09-30,320000.0
139550,Exempt,5600 W FOND DU LAC AV,7,24910,Commercial Exempt,None,None,None,None,None,0,None,0,0,0.0,2022-09-09,225000.0
172821,Residential,3063 N HOLTON ST,6,2930,None,None,None,None,None,None,0,None,0,0,3136.0,2022-04-22,10000.0
175501,Residential,2626-2628 N BARTLETT AV,3,3190,None,None,None,None,None,None,0,None,0,0,3600.0,2022-08-31,75000.0
177302,Commercial,2669-2671 N MARTIN L KING JR DR,6,6258,Office Building - Multi Story ( Ofc & Apt),None,None,None,None,0.0,1,None,0,0,10500.0,2022-01-20,347000.0
205735,Commercial,7225 W MAIN ST,10,6272,Parking Lot,None,None,None,None,0.0,300,None,0,0,141352.0,2022-09-13,850000.0


In [19]:
%%sql
-- Delete rows missing Year_Built (consider saving them first if you need to audit)
DELETE FROM sales_staging
WHERE Year_Built IS NULL OR
TRIM(Year_Built) ='';

 * sqlite:///real_estate_trends.db
11 rows affected.


[]

In [20]:
%%sql
SELECT * FROM sales_staging
WHERE Sale_price IS NULL
OR Sale_price ='';

 * sqlite:///real_estate_trends.db
Done.


PropertyID,PropType,Address,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price


In [21]:
%%sql
SELECT * FROM sales_staging
WHERE FinishedSqft IS NULL
OR FinishedSqft ='';

 * sqlite:///real_estate_trends.db
Done.


PropertyID,PropType,Address,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
261051,Residential,8072 N 107TH ST,9,240,None,None,0,0,None,None,0,None,0,0,24910.0,2022-03-31,20000.0
104919,Residential,7235 N GRANVILLE RD,5,440,None,None,0,0,None,None,0,None,0,0,87120.0,2022-10-19,145000.0
105063,Residential,10445 W JUNIPER ST,5,440,None,None,0,0,None,None,0,None,0,0,47158.0,2022-05-31,37500.0
143109,Residential,3523 N 93RD ST,5,2100,None,None,0,0,None,None,0,None,0,0,9120.0,2022-05-20,25000.0
165541,Residential,2944 N 58TH ST,10,1860,None,None,0,0,None,None,0,None,0,0,3880.0,2022-07-14,10000.0
167839,Residential,2869 N 38TH ST,15,1780,None,None,0,0,None,None,0,None,0,0,4800.0,2022-12-02,7000.0
168844,Residential,2954-2956 N 24TH ST,7,2950,None,None,0,0,None,None,0,None,0,0,3750.0,2022-05-25,4000.0
188450,Residential,2221 N 5TH ST,6,2970,None,None,0,0,None,None,0,None,0,0,5572.0,2022-03-15,26000.0
196143,Residential,1321-1323 N 37TH PL,15,2380,None,None,0,0,None,None,0,None,0,0,4945.0,2022-02-28,9000.0
208213,Residential,707-709 S 4TH ST,12,4020,None,None,0,0,None,None,0,None,0,0,7000.0,2022-10-13,105000.0


In [22]:
%%sql
DELETE FROM sales_staging
WHERE FinishedSqft IS NULL
OR FinishedSqft ='';

 * sqlite:///real_estate_trends.db
17 rows affected.


[]

In [23]:
%%sql
SELECT COUNT(*) AS total_rows FROM sales_staging 
WHERE Rooms IS NULL
OR Rooms ='';

 * sqlite:///real_estate_trends.db
Done.


total_rows
415


In [25]:
%%sql
SELECT * FROM sales_staging 
WHERE Rooms IS NULL
OR Rooms ='' LIMIT 10;

 * sqlite:///real_estate_trends.db
Done.


PropertyID,PropType,Address,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
98461,Manufacturing,9434-9446 N 107TH ST,9,6300,Service Building,Concrete Block,1,1978,None,20600.0,6,None,0,0,0.0,2022-04-01,950000.0
98464,Commercial,9306-9316 N 107TH ST,9,6202,Office Building - 1 Story,Brick,1,1982,None,9688.0,23,None,0,0,35719.0,2022-10-07,385000.0
98976,Commercial,7400 W BROWN DEER RD,9,6202,Office Building - Multi Story ( Ofc & Apt),Concrete Block,2,1978,None,7444.0,4,None,0,0,0.0,2022-09-13,475000.0
100122,Residential,11503 W HEATHER DR,9,240,Tudor,Fiber Cement/Hardiplank,1,2022,None,2587.0,1,None,2,1,12187.0,2022-07-19,47400.0
100307,Commercial,8687 N 107TH ST,9,6420,Warehouse Building - 1 Story,Concrete Block,1,1930,None,49133.0,7,None,0,0,231739.0,2022-12-29,1940000.0
100627,Commercial,8525 N 87TH ST,9,6453,Office Warehouse Park - multi tenants,Concrete Block,1,1988,None,10027.0,1,None,0,0,0.0,2022-11-16,650000.0
100629,Manufacturing,8800 W DEAN RD,9,6300,Warehouse Building - 1 Story,Metal Siding,1,1989,None,20860.0,1,None,0,0,0.0,2022-06-15,1525000.0
100873,Commercial,8301 W BROWN DEER RD,9,6202,"Store Building - Single tenant, 1 story",Concrete Block,1,1980,None,1703.0,1,None,0,0,0.0,2022-01-28,385000.0
100874,Commercial,7600 W DEAN RD,9,6206,Office Building - 1 Story,Fiber Cement/Hardiplank,1,1975,None,5567.0,1,None,0,0,41991.0,2022-09-30,275000.0
101089,Commercial,8111 W BROWN DEER RD,9,6202,"Store Building - Single tenant, 1 story",None,1,2004,None,3871.0,1,None,0,0,29293.0,2022-06-10,425000.0


In [26]:
%%sql
DELETE FROM sales_staging
WHERE Rooms IS NULL
OR Rooms = 'None';

 * sqlite:///real_estate_trends.db
415 rows affected.


[]

In [27]:
%%sql
SELECT * FROM sales_staging
WHERE nbhd IS NULL;

 * sqlite:///real_estate_trends.db
Done.


PropertyID,PropType,Address,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price


In [28]:
%%sql
SELECT * FROM sales_staging LIMIT 10;

 * sqlite:///real_estate_trends.db
Done.


PropertyID,PropType,Address,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
98519,Residential,9411 W COUNTY LINE RD,9,40,Ranch,Aluminum/Vinyl,1,1959,6,1334.0,1,3,1,1,83200.0,2022-08-09,280000.0
98561,Residential,9322 N JOYCE AV,9,40,Ranch,Aluminum/Vinyl,1,1980,10,1006.0,1,6,1,0,8303.0,2022-05-23,233100.0
98593,Residential,9360 N 85TH ST,9,40,Ranch,Aluminum/Vinyl,1,1982,5,1007.0,1,3,1,0,7200.0,2022-07-25,215000.0
98604,Residential,9305 N BURBANK AV,9,40,Ranch,Aluminum/Vinyl,1,1984,5,1301.0,1,3,2,0,7200.0,2022-03-29,150000.0
98608,Residential,9217 N 83RD ST,9,40,Colonial,Aluminum/Vinyl,2,2007,9,2237.0,1,4,2,1,15677.0,2022-05-10,400000.0
98696,Condominium,"9192 N 70TH ST, Unit 2",9,5010,Condo Townhouse,None,2,1973,7,1437.0,1,3,2,1,0.0,2022-05-16,122000.0
98715,Condominium,"9212 N 70TH ST, Unit 8",9,5010,Condo Townhouse,None,2,1973,7,1437.0,1,4,2,1,0.0,2022-04-14,123000.0
98717,Condominium,"9242 N 70TH ST, Unit 2",9,5010,Condo Townhouse,None,2,1973,7,1437.0,1,4,2,1,0.0,2022-03-11,106000.0
98719,Condominium,"9238 N 70TH ST, Unit 4",9,5010,Condo Ranch,None,1,1973,4,898.0,1,2,1,1,0.0,2022-07-07,95000.0
98723,Condominium,"9230 N 70TH ST, Unit 8",9,5010,Condo Townhouse,None,1,1973,10,1437.0,1,6,2,1,0.0,2022-02-04,122000.0


### 9) Read cleaned table back into pandas and export
After SQL-based cleaning, I am converting the final `sales_staging` table back into a pandas dataframe and export the cleaned data to CSV for Tableau.

In [29]:
df = pd.read_sql('SELECT * FROM sales_staging;', engine)
print("Cleaned data shape:", df.shape)
df.head()

Cleaned data shape: (6701, 18)


,PropertyID,PropType,Address,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
0,98519,Residential,9411 W COUNTY LINE RD,9,40,Ranch,Aluminum/Vinyl,1.0,1959,6,1334.0,1,3,1,1,83200.0,2022-08-09,280000.0
1,98561,Residential,9322 N JOYCE AV,9,40,Ranch,Aluminum/Vinyl,1.0,1980,10,1006.0,1,6,1,0,8303.0,2022-05-23,233100.0
2,98593,Residential,9360 N 85TH ST,9,40,Ranch,Aluminum/Vinyl,1.0,1982,5,1007.0,1,3,1,0,7200.0,2022-07-25,215000.0
3,98604,Residential,9305 N BURBANK AV,9,40,Ranch,Aluminum/Vinyl,1.0,1984,5,1301.0,1,3,2,0,7200.0,2022-03-29,150000.0
4,98608,Residential,9217 N 83RD ST,9,40,Colonial,Aluminum/Vinyl,2.0,2007,9,2237.0,1,4,2,1,15677.0,2022-05-10,400000.0


In [ ]:

df.to_csv(os.path.join(os.getcwd(),'RealEstate.csv'), index=False)
print("Exported RealEstate.csv")

Exported RealEstate.csv


In [ ]:
# Exporting addresses for geocoding or mapping
address_df = pd.read_sql('SELECT Address FROM sales_staging;', engine)
address_df.to_csv(os.path.join(os.getcwd(), 'address.csv'), index=False)
print("Exported address.csv")

Exported address.csv
